# Рекомендательные системы. Рекомендации через поиск ближайших соседей

> На этом практическом занятии мы с вами сделаем следующее:
- Посмотри как работает FAISS.
- Построим простейший сервис для рекомендаций используя FAISS.

## FAISS

> Необходимо сперва установить faiss библиотеку. Инструкцию можно найти [здесь](https://github.com/facebookresearch/faiss/blob/master/INSTALL.md).
```conda install faiss-cpu -c pytorch # CPU version only```

In [16]:
import numpy as np

In [59]:
dim = 512  # рассмотрим произвольные векторы размерности 512
nb = 10000  # количество векторов в индексе
nq = 1 # количество векторов в выборке для поиска
np.random.seed(58) # DON't CHANGE THIS
vectors = np.random.random((nb, dim)).astype('float32')
query = np.random.random((nq, dim)).astype('float32')

In [44]:
vectors.shape

(10000, 512)

In [60]:
vectors

array([[0.36510557, 0.4512059 , 0.49606034, ..., 0.76589304, 0.00648978,
        0.83173156],
       [0.4488682 , 0.98864484, 0.9459802 , ..., 0.3300889 , 0.31562248,
        0.37883386],
       [0.85288   , 0.15087937, 0.48739725, ..., 0.4501739 , 0.6276719 ,
        0.9150207 ],
       ...,
       [0.5569578 , 0.4969434 , 0.5190067 , ..., 0.04685995, 0.11529469,
        0.6037052 ],
       [0.9910725 , 0.42323965, 0.31499565, ..., 0.8129928 , 0.57980275,
        0.84211975],
       [0.33896688, 0.8154824 , 0.9685506 , ..., 0.32354274, 0.49578118,
        0.55211055]], dtype=float32)

### IndexFlatL2

Создаем Flat индекс и добавляем векторы без обучения

In [46]:
import faiss # предварительно необходимо установить FAISS см. выше


index = faiss.IndexFlatL2(dim)
index.add(vectors)
print(index.ntotal)

10000


Проведем поиск по нашим векторам из query:

In [49]:
%%time

top_7 = 7
D, I = index.search(query, top_7)

print(I)
print(D)

[[3214 8794 9507 6591 8728 3959 5485]]
[[70.102005 70.75225  72.44308  72.87785  72.94414  73.420784 73.756744]]
CPU times: user 4.91 ms, sys: 1.74 ms, total: 6.65 ms
Wall time: 3.93 ms


### Inverted File Index

Необходмио создать quantiser(IndexFlatL2), индекс (IndexIVFFlat), обучить индекс и добавить вектора в индекс.

In [61]:
%%time
k = 10 # количество центроидов

quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantiser, dim, k)
index.train(vectors)
index.add(vectors)

CPU times: user 45.1 ms, sys: 18.3 ms, total: 63.4 ms
Wall time: 36.9 ms


Необходимо произвести поиск по индексу нашего запроса (query).

In [72]:
%%time

top_7 = 7

D, I = index.search(query, top_7)
print(I)
print(D)

[[5485 9530 8678 7046 5314 3492 4042]]
[[73.756744 74.0772   74.20973  75.07443  75.194534 75.36259  75.704254]]
CPU times: user 2.91 ms, sys: 1.59 ms, total: 4.49 ms
Wall time: 3.32 ms


## Применим FAISS для рекомендаций в нашей задаче

Построим простейший рекомендательный сервис.

In [29]:
import faiss
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from flask import Flask, jsonify, request

# constants
RANDOM_STATE = 57
N_FACTOR = 20 # размерность эмбедингов
N_RESULT = 10 # сколько фильмов рекомендуем

In [14]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [17]:
users = sorted(np.unique(ratings['userId']))
movies = sorted(np.unique(ratings['movieId']))

In [19]:
# for later use
user_id2i = {id: i for i, id in enumerate(users)}
movie_id2i = {id: i for i, id in enumerate(movies)}
movie_i2id = {i: id for i, id in enumerate(movies)}

In [24]:
# make sparse matrix
rating_mat = coo_matrix(
    (ratings['rating'], (ratings['userId'].map(user_id2i), ratings['movieId'].map(movie_id2i)))
)

In [27]:
rating_mat.todense()

matrix([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [31]:
# decompose
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_mat = model.fit_transform(rating_mat)
movie_mat = model.components_.T

/Users/evgenijgrinev/mambaforge/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


> **NMF** = Non-negative Matrix Factorization. Можно применять метод чередующихся наименьших квадратов (ALS) для неотрицательного матричного разложения. Ключевая идея - искать поочередно то столбцы $p_t$, то столбцы $q_t$ при фиксированных остальных.

In [33]:
# indexing
# movie_index = faiss.IndexFlatL2(N_FACTOR)

########################
# YOUR CODE HERE
########################
k = 100 # количество центроидов
# необходимо дописать методы
quantiser = faiss.IndexFlatL2(N_FACTOR)
movie_index = faiss.IndexIVFFlat(quantiser, dim, k)
movie_index.train(movie_mat)
movie_index.add(movie_mat)

RuntimeError: Error in faiss::IndexIVF::IndexIVF(faiss::Index *, size_t, size_t, size_t, faiss::MetricType) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/IndexIVF.cpp:175: Error: 'd == quantizer->d' failed

In [36]:
# create app
app = Flask(__name__)

In [40]:
# API endpoint
@app.route('/')
def recom_for_user():
    user_id = request.args.get('user_id', default = 1, type = int)
    user_i = user_id2i[user_id]
    
    ########################
    # YOUR CODE HERE
    ########################
    # необходимо определить вектор пользователя (пользовательский эмбединг) и найти ближайшие к этому вектору индексы из фильмов
    user_vec = user_mat[user_i].astype('float32')
    scores, indices = movie_index.search(np.array([user_vec]), N_RESULT)
    
    
    movie_scores = zip(indices[0], scores[0])
    return jsonify(
        movies=[
            {
                "id": int(movie_i2id[i]),
                "score": float(s),
            }
            for i, s in movie_scores
        ],
    )

AssertionError: View function mapping is overwriting an existing endpoint function: recom_for_user

In [ ]:
app.run(host="0.0.0.0", port=5000)

> Note: use this link in your browser to acccess your server: http://0.0.0.0:5000/?user_id=128